# Linear Regression have two methodolgy to solve this problem
* [一些基本知識](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html)
* Normal Equation
* Gradient Descent (ToDo)


In [55]:
from sklearn import metrics
import numpy as np
import pandas as pd

In [8]:
# ?load_boston

In [6]:
from sklearn.datasets import load_boston
X, y = load_boston(return_X_y=True)
print(X.shape)
print(y.shape)

(506, 13)
(506,)


# Normal Equation(do it by native numpy code)
<img width=20% src="https://i.imgur.com/aZuHRKU.png" />
* the objective is to minimize the below square error
<img width=20% src="https://i.imgur.com/uuj1z1y.jpg" />
* x 為 training data as m*n matrix , m is the numboer of sample . n is the number of feature size.
* user above function could find the best parameter theta, it could minimize the squre error. 
* so this method is called as "Least Squares"

In [84]:
# 沒有加常數項的情況
theta = np.linalg.inv((X.T).dot(X)).dot(X.T).dot(y)

# 加上常數項
X_new = np.append(X,np.ones(X.shape[0]).reshape([-1,1]) ,axis=1)
theta_new = np.linalg.inv((X_new.T).dot(X_new)).dot(X_new.T).dot(y)
theta_new

array([-1.08011358e-01,  4.64204584e-02,  2.05586264e-02,  2.68673382e+00,
       -1.77666112e+01,  3.80986521e+00,  6.92224640e-04, -1.47556685e+00,
        3.06049479e-01, -1.23345939e-02, -9.52747232e-01,  9.31168327e-03,
       -5.24758378e-01,  3.64594884e+01])

In [27]:
def linear_regression(input_X):
    return input_X.dot(theta)
    

In [48]:
predict_y = linear_regression(X)

In [59]:
theta0 = (predict_y-y)
theta0.mean()

-2.5276223801742696e-16

In [49]:
metrics.mean_squared_error(y,predict_y)

24.166099330126492

In [50]:
metrics.mean_absolute_error(y,predict_y)

3.2867335789615058

# Scikit Learn 內建的方法
* 請參考 [sklearn](https://github.com/scikit-learn/scikit-learn/blob/95119c13a/sklearn/linear_model/_base.py#L391) 裡面的程式碼
* 請參考 [scipy](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.nnls.html)

In [51]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression().fit(X, y)


In [52]:
predict_y = reg.predict(X)

In [53]:
metrics.mean_squared_error(y,predict_y)

21.894831181729202

In [39]:
metrics.mean_absolute_error(y,predict_y)

3.270862810900316

# Compare Theta

In [46]:
theta

array([-9.28965170e-02,  4.87149552e-02, -4.05997958e-03,  2.85399882e+00,
       -2.86843637e+00,  5.92814778e+00, -7.26933458e-03, -9.68514157e-01,
        1.71151128e-01, -9.39621540e-03, -3.92190926e-01,  1.49056102e-02,
       -4.16304471e-01])

In [45]:
reg.coef_

array([-1.08011358e-01,  4.64204584e-02,  2.05586264e-02,  2.68673382e+00,
       -1.77666112e+01,  3.80986521e+00,  6.92224640e-04, -1.47556685e+00,
        3.06049479e-01, -1.23345939e-02, -9.52747232e-01,  9.31168327e-03,
       -5.24758378e-01])

In [60]:
reg.intercept_

36.459488385090125

# 處理 Collinearity 問題
* 當 Feature 之間如果有 dependency，在用 normal equation 來求解 theta 可能會太多解。


In [14]:
df_X.shape

(506, 13)

In [12]:
df_X = pd.DataFrame(X)
corr = df_X.corr()

In [15]:
# corr=house_selected.corr()
corr.style.background_gradient(cmap='coolwarm')

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,1.000000,-0.200469,0.406583,-0.055892,0.420972,-0.219247,0.352734,-0.379670,0.625505,0.582764,0.289946,-0.385064,0.455621
1,-0.200469,1.000000,-0.533828,-0.042697,-0.516604,0.311991,-0.569537,0.664408,-0.311948,-0.314563,-0.391679,0.175520,-0.412995
2,0.406583,-0.533828,1.000000,0.062938,0.763651,-0.391676,0.644779,-0.708027,0.595129,0.720760,0.383248,-0.356977,0.603800
3,-0.055892,-0.042697,0.062938,1.000000,0.091203,0.091251,0.086518,-0.099176,-0.007368,-0.035587,-0.121515,0.048788,-0.053929
4,0.420972,-0.516604,0.763651,0.091203,1.000000,-0.302188,0.731470,-0.769230,0.611441,0.668023,0.188933,-0.380051,0.590879
5,-0.219247,0.311991,-0.391676,0.091251,-0.302188,1.000000,-0.240265,0.205246,-0.209847,-0.292048,-0.355501,0.128069,-0.613808
6,0.352734,-0.569537,0.644779,0.086518,0.731470,-0.240265,1.000000,-0.747881,0.456022,0.506456,0.261515,-0.273534,0.602339
7,-0.379670,0.664408,-0.708027,-0.099176,-0.769230,0.205246,-0.747881,1.000000,-0.494588,-0.534432,-0.232471,0.291512,-0.496996
8,0.625505,-0.311948,0.595129,-0.007368,0.611441,-0.209847,0.456022,-0.494588,1.000000,0.910228,0.464741,-0.444413,0.488676
9,0.582764,-0.314563,0.720760,-0.035587,0.668023,-0.292048,0.506456,-0.534432,0.910228,1.000000,0.460853,-0.441808,0.543993


## 如果把 row number 給予降低，就可能發生 singular matrix 
* singular 定義: https://zh.wikipedia.org/wiki/%E9%9D%9E%E5%A5%87%E5%BC%82%E6%96%B9%E9%98%B5

## Singular Matirx 直觀的解釋[摘自](https://ccjou.wordpress.com/2010/03/12/%E5%8F%AF%E9%80%86%E7%9F%A9%E9%99%A3%E5%AE%9A%E7%90%86/)

* <img src="https://i.imgur.com/BqlQzol.png" width=50% />

## 遇到這種狀況
* 去掉 high correlation feature pair
* 增加 rows
* 不用使用 normal equation ，白話的說不要使用 l2 做為 objective function，改用 L1，例如
* LASSO :[L1 Classifier](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html)



In [42]:
#theta = np.linalg.inv((X.T).dot(X)).dot(X.T).dot(y)

X_small = X[:5]
y_small = y[:5]

if np.linalg.det((X_small.T).dot(X_small))==0 : print("this is matrix is singular")

try:
    np.linalg.inv((X_small.T).dot(X_small))
except:
    print("的確解不出來")

from scipy.linalg import pinv
# 利用 pseudo-inverse 就可以解的出來
ii = pinv((X_small.T).dot(X_small))
print("利用 pseudo-inverse 就可以解的出來??\n",ii)

this is matrix is singular
的確解不出來
利用 pseudo-inverse 就可以解的出來??
 [[ 4.97833124e-02  6.39340649e-02 -3.46781928e-03  2.05874888e-15
  -2.89234506e-04  6.82434268e-01  2.82553258e-02 -1.42866679e-02
  -1.05935830e-02 -1.87289223e-02 -3.51528513e-02 -3.07642444e-03
  -2.00973686e-02]
 [ 6.39340649e-02  9.78393008e-02 -1.95187749e-02  2.64235279e-15
  -3.51193076e-04  8.73806848e-01  4.01412493e-02 -1.68964555e-02
  -1.23667693e-02 -2.87561838e-02 -4.59772249e-02 -1.67858184e-03
  -1.72430884e-02]
 [-3.46781928e-03 -1.95187749e-02  1.46907704e-02 -1.41881848e-16
   7.77078638e-07 -4.50361529e-02 -5.67146852e-03 -3.99875009e-04
  -4.52409012e-04  5.88981813e-03  3.24048289e-03 -2.00978569e-03
  -6.82210084e-03]
 [ 5.72792139e-15  7.33332665e-15 -3.76434994e-16  2.36732396e-28
  -3.33070296e-17  7.85072284e-14  3.24111397e-15 -1.64481601e-15
  -1.22078454e-15 -2.14953923e-15 -4.04244780e-15 -3.57515666e-16
  -2.32463788e-15]
 [-2.89234506e-04 -3.51193076e-04  7.77078641e-07 -1.19630925e-17
   

# Gradient Descent(TBD) 找個時間再來實作吧！！！
* 使用 gradient descent 來求解